In [1]:
import pandas as pd


In [2]:
from keras.datasets import imdb


Using TensorFlow backend.


In [3]:
reviews = pd.read_csv('employee_reviews.csv')

In [4]:
reviews["prosandcons"]=reviews["pros"]+". "+reviews["cons"]

In [5]:
#Use data that can show you a good results Print out 
#the data after each data to 
#see the current results

In [6]:
def getsentiment(rating):
    if int(rating) > 3:
        return 1
    if int(rating) <= 3:
        return 0
    

In [7]:
reviews["sentiment"]=reviews["overall-ratings"].apply(getsentiment)

In [8]:
data = reviews[['prosandcons','sentiment']]

In [9]:
data

,prosandcons,sentiment
0,People are smart and friendly. Bureaucracy is ...,1
1,"1) Food, food, food. 15+ cafes on main campus ...",1
2,"* If you're a software engineer, you're among ...",1
3,You can't find a more well-regarded company th...,1
4,Google is a world of its own. At every other c...,1
5,"People are not that busy, so they are nice to ...",1
6,Great working environment. Good work life bala...,1
7,"work culture, benefits, growth, people,. No co...",1
8,Great products. Vision you can feel good about...,1
9,Impact driven. Best tech in the world.. Size m...,1


In [10]:
X =  data['prosandcons']

y = data['sentiment']

In [11]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X) 
sequences = tokenizer.texts_to_sequences(X)


word_index = tokenizer.word_index
print('unique words %s' % len(word_index))

stupiddata = pad_sequences(sequences, maxlen=500, padding='post')
print('data shape:', stupiddata.shape)



unique words 38357
data shape: (67529, 500)


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(stupiddata,y , test_size=0.2, random_state=42)

In [13]:
# print('Maximum review length: {}'.format(
# len(max((X_train + X_test), key=len))))

In [14]:
# print('Minimum review length: {}'.format(
# len(min((X_test + X_test), key=len))))

In [15]:
# from keras.preprocessing import sequence
# max_words = 500
# X_train = sequence.pad_sequences(X_train, maxlen=max_words)
# X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [16]:
#Glove Vectors loaded
import numpy as np

embeddings_index = {}

f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:

    values = line.split()

    word = values[0]

    coefs = np.asarray(values[1:], dtype='float32')

    embeddings_index[word] = coefs

f.close()

 

print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [17]:
#do not need to do pipelining

#want to use the model.fit- parse the train data and then specify the epochs and batch size.


# prepare embeddings for layer
from keras.layers.embeddings import Embedding

words_max = len(word_index)+1

embedding_matrix = np.zeros((len(word_index)+1, 100))

for word, i in word_index.items():

    if i >= words_max:

        continue

    embedding_vector = embeddings_index.get(word)

    if embedding_vector is not None:

        embedding_matrix[i] = embedding_vector

In [18]:
##this is where add Karuna Code
# Create an non re-trainable embedding layer with embedding size 100

nb_words = len(word_index)+1

embedding_layer = Embedding(nb_words,100,

        weights=[embedding_matrix],

        input_length=500,

        trainable=True)

In [23]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
embedding_size=32
model=Sequential()
model.add(embedding_layer)
# model.add(Embedding(5000, embedding_size, input_length=500))
model.add(LSTM(100))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          3835800   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 3,916,301
Trainable params: 3,916,301
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [25]:
batch_size = 75
num_epochs = 17

X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)

Instructions for updating:
Use tf.cast instead.
Train on 53948 samples, validate on 75 samples
Epoch 1/17
53948/53948 [==============================] - 665s 12ms/step - loss: 0.6293 - acc: 0.6795 - val_loss: 0.5798 - val_acc: 0.7333
Epoch 2/17
53948/53948 [==============================] - 691s 13ms/step - loss: 0.6277 - acc: 0.6801 - val_loss: 0.5837 - val_acc: 0.7333
Epoch 3/17
53948/53948 [==============================] - 800s 15ms/step - loss: 0.6289 - acc: 0.6795 - val_loss: 0.5990 - val_acc: 0.7333
Epoch 4/17
53948/53948 [==============================] - 724s 13ms/step - loss: 0.6250 - acc: 0.6812 - val_loss: 0.5606 - val_acc: 0.7467
Epoch 5/17
53948/53948 [==============================] - 740s 14ms/step - loss: 0.6229 - acc: 0.6831 - val_loss: 0.5533 - val_acc: 0.7600
Epoch 6/17
53948/53948 [==============================] - 690s 13ms/step - loss: 0.6180 - acc: 0.6842 - val_loss: 0.5884 - val_acc: 0.7333
Epoch 7/17
53948/53948 [==============================] - 670s 12ms/ste

In [26]:
scores = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])


Test accuracy: 0.7437435213978972


In [ ]:
#Batch 2 with 2 LSTMS

In [20]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
embedding_size=32
model=Sequential()
model.add(embedding_layer)
# model.add(Embedding(5000, embedding_size, input_length=500))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(32))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          3835800   
_________________________________________________________________
lstm_5 (LSTM)                (None, 500, 100)          80400     
_________________________________________________________________
lstm_6 (LSTM)                (None, 32)                17024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 3,933,257
Trainable params: 3,933,257
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
model.compile(loss='mean_squared_error', 
             optimizer='adam', 
             metrics=['accuracy'])

In [22]:
batch_size = 75
num_epochs = 17

X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)

Instructions for updating:
Use tf.cast instead.
Train on 53948 samples, validate on 75 samples
Epoch 1/17
53948/53948 [==============================] - 1332s 25ms/step - loss: 0.2188 - acc: 0.6794 - val_loss: 0.2057 - val_acc: 0.7200
Epoch 2/17
53948/53948 [==============================] - 2298s 43ms/step - loss: 0.2196 - acc: 0.6773 - val_loss: 0.1944 - val_acc: 0.7333
Epoch 3/17
53948/53948 [==============================] - 1004s 19ms/step - loss: 0.2152 - acc: 0.6842 - val_loss: 0.1910 - val_acc: 0.7467
Epoch 4/17
53948/53948 [==============================] - 988s 18ms/step - loss: 0.2140 - acc: 0.6859 - val_loss: 0.1860 - val_acc: 0.7467
Epoch 5/17
53948/53948 [==============================] - 1010s 19ms/step - loss: 0.2121 - acc: 0.6833 - val_loss: 0.1859 - val_acc: 0.7467
Epoch 6/17
53948/53948 [==============================] - 1003s 19ms/step - loss: 0.2179 - acc: 0.6802 - val_loss: 0.1973 - val_acc: 0.7333
Epoch 7/17
53948/53948 [==============================] - 1010s 19

In [23]:
scores = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])


Test accuracy: 0.7701762179771953


In [ ]:
#Stacked LSTM MODEL

In [30]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
embedding_size=32
model=Sequential()
model.add(embedding_layer)
# model.add(Embedding(5000, embedding_size, input_length=500))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))

model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          3835800   
_________________________________________________________________
lstm_23 (LSTM)               (None, 500, 100)          80400     
_________________________________________________________________
lstm_24 (LSTM)               (None, 500, 32)           17024     
_________________________________________________________________
lstm_25 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 3,941,577
Trainable params: 3,941,577
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(loss='mean_squared_error', 
             optimizer='adam', 
             metrics=['accuracy'])

In [33]:
batch_size = 75
num_epochs = 34

X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)

Train on 53948 samples, validate on 75 samples
Epoch 1/17
53948/53948 [==============================] - 48644s 902ms/step - loss: 0.2196 - acc: 0.6774 - val_loss: 0.2038 - val_acc: 0.7200
Epoch 2/17
53948/53948 [==============================] - 1465s 27ms/step - loss: 0.2189 - acc: 0.6779 - val_loss: 0.2027 - val_acc: 0.7200
Epoch 3/17
53948/53948 [==============================] - 1891s 35ms/step - loss: 0.2173 - acc: 0.6797 - val_loss: 0.1956 - val_acc: 0.7333
Epoch 4/17
53948/53948 [==============================] - 1598s 30ms/step - loss: 0.2154 - acc: 0.6852 - val_loss: 0.2019 - val_acc: 0.7333
Epoch 5/17
53948/53948 [==============================] - 1702s 32ms/step - loss: 0.2170 - acc: 0.6819 - val_loss: 0.1976 - val_acc: 0.7333
Epoch 6/17
53948/53948 [==============================] - 6785s 126ms/step - loss: 0.1956 - acc: 0.7266 - val_loss: 0.1827 - val_acc: 0.7600
Epoch 7/17
53948/53948 [==============================] - 14966s 277ms/step - loss: 0.1823 - acc: 0.7591 - val

In [34]:
scores = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])


Test accuracy: 0.7449281800681179


In [63]:
#3rd Model with 3 LSTMS More Epochs and Keras Callback early stopping  - did not improve - too early
# Note min delta .1 too early , changed to .6 to see if can improve - did not improve - too early
# Note min delta .6 too early , changed to 5 to see if can improve - Did not improve - too early
# Note min delta .6 too early , changed patience from 2 to 3.5 and kept min delta at .005


In [19]:
from keras import Sequential, callbacks
from keras.layers import Embedding, LSTM, Dense, Dropout
es = [callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3.5, min_delta=.005)]
embedding_size=32
model=Sequential()
model.add(embedding_layer)
# model.add(Embedding(5000, embedding_size, input_length=500))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))

model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          3835800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 500, 100)          80400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 500, 32)           17024     
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)      

In [20]:
model.compile(loss='mean_squared_error', 
             optimizer='adam', 
             metrics=['accuracy'])

In [21]:
batch_size = 75
num_epochs = 25

X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs, callbacks=es)

Instructions for updating:
Use tf.cast instead.
Train on 53948 samples, validate on 75 samples
Epoch 1/25
53948/53948 [==============================] - 2263s 42ms/step - loss: 0.2189 - acc: 0.6796 - val_loss: 0.1998 - val_acc: 0.7333
Epoch 2/25
53948/53948 [==============================] - 2244s 42ms/step - loss: 0.2200 - acc: 0.6762 - val_loss: 0.2041 - val_acc: 0.7200
Epoch 3/25
53948/53948 [==============================] - 2217s 41ms/step - loss: 0.2188 - acc: 0.6779 - val_loss: 0.2056 - val_acc: 0.7200
Epoch 4/25
53948/53948 [==============================] - 2203s 41ms/step - loss: 0.2187 - acc: 0.6779 - val_loss: 0.2039 - val_acc: 0.7200
Epoch 5/25
53948/53948 [==============================] - 2240s 42ms/step - loss: 0.2187 - acc: 0.6779 - val_loss: 0.2027 - val_acc: 0.7200
Epoch 00005: early stopping


In [22]:
scores = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])


Test accuracy: 0.6711831778468829


In [ ]:
#New

In [19]:
from keras import Sequential, callbacks
from keras.layers import Embedding, LSTM, Dense, Dropout
es = [callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3.2, min_delta=.01)]
embedding_size=32
model=Sequential()
model.add(embedding_layer)
# model.add(Embedding(5000, embedding_size, input_length=500))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))

model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          3835800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 500, 100)          80400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 500, 32)           17024     
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)      

In [20]:
model.compile(loss='mean_squared_error', 
             optimizer='adam', 
             metrics=['accuracy'])

In [ ]:
batch_size = 75
num_epochs = 25

X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs, callbacks=es)

Instructions for updating:
Use tf.cast instead.
Train on 53948 samples, validate on 75 samples
Epoch 1/25
53948/53948 [==============================] - 1458s 27ms/step - loss: 0.2188 - acc: 0.6796 - val_loss: 0.1990 - val_acc: 0.7333
Epoch 2/25
53948/53948 [==============================] - 1455s 27ms/step - loss: 0.2182 - acc: 0.6802 - val_loss: 0.1973 - val_acc: 0.7333
Epoch 3/25
53948/53948 [==============================] - 1782s 33ms/step - loss: 0.2180 - acc: 0.6802 - val_loss: 0.1982 - val_acc: 0.7333
Epoch 4/25
14325/53948 [======>.......................] - ETA: 17:49 - loss: 0.2165 - acc: 0.6840